In [1]:
import pandas as pd
import os

In [2]:
data = pd.read_csv('./Thomas/data_quarterly_DE.csv')
data['date'] = pd.PeriodIndex(data['date'], freq= 'Q').to_timestamp()
data

,iso2,date,date.1,loansPnfs_yoy,resPropPrice,cpi_yoy,fx,totalCreditPnfs2GDP,totalCreditPnfsLCY,dsrPnfs,...,totalCreditPnfs2GDP_lag2,cpi_yoy_lag3,bankCreditPnfs_lag3,totalCreditPnfsLCY_lag3,totalCreditPnfs2GDP_lag3,cpi_yoy_lag4,bankCreditPnfs_lag4,totalCreditPnfsLCY_lag4,totalCreditPnfs2GDP_lag4,fx_std
0,DE,1948-10-01,1948-10-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,4.522,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DE,1949-01-01,1949-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,5.476,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DE,1949-04-01,1949-04-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,6.942,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DE,1949-07-01,1949-07-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,8.593,NaN,...,NaN,NaN,2.984000,4.522,NaN,NaN,NaN,NaN,NaN,NaN
4,DE,1949-10-01,1949-10-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,10.573,NaN,...,NaN,NaN,3.614000,5.476,NaN,NaN,2.984000,4.522,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,DE,2023-04-01,2023-05-16 01:49:05.454545408,NaN,133.1571,6.796917,0.918634,125.1,5003.808,10.8,...,128.1,7.110009,2115.533000,4926.568,129.4,6.494617,2128.664667,4817.403,128.4,0.089809
299,DE,2023-07-01,2023-08-13 23:17:38.823529472,NaN,130.5617,5.832606,0.918989,123.5,5020.222,10.8,...,126.3,8.676878,2215.088333,4957.597,128.1,7.110009,2115.533000,4926.568,129.4,0.043293
300,DE,2023-10-01,2023-11-11 17:27:16.363636480,NaN,127.8992,3.648422,0.930387,122.0,5028.972,10.8,...,125.1,8.406074,2243.533667,4972.174,126.3,8.676878,2215.088333,4957.597,128.1,0.053981
301,DE,2024-01-01,2024-02-13 07:30:00.000000000,NaN,NaN,2.519272,0.921022,NaN,NaN,NaN,...,123.5,6.796917,2251.737333,5003.808,125.1,8.406074,2243.533667,4972.174,126.3,0.023914


In [3]:
dummy = pd.read_csv('dummy_final.csv')
dummy_de = dummy[dummy['iso2']=='DE']
dummy_de['date'] = pd.to_datetime(dummy_de['date.1'])
dummy_de

C:\Users\Thoma\AppData\Local\Temp\ipykernel_23044\1298440593.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_de['date'] = pd.to_datetime(dummy_de['date.1'])


,iso2,date.1,is_systemic_crisis,date
343,DE,1974-06-01,1.0,1974-06-01
344,DE,1974-07-01,1.0,1974-07-01
345,DE,1974-08-01,1.0,1974-08-01
346,DE,1974-09-01,1.0,1974-09-01
347,DE,1974-10-01,1.0,1974-10-01
...,...,...,...,...
5908,DE,2023-11-01,0.0,2023-11-01
5909,DE,2023-12-01,0.0,2023-12-01
5910,DE,2024-01-01,0.0,2024-01-01
5911,DE,2024-02-01,0.0,2024-02-01


In [4]:
data = pd.merge(data, dummy_de, on = ['iso2', 'date'], how= 'left')

In [5]:
data

,iso2,date,date.1_x,loansPnfs_yoy,resPropPrice,cpi_yoy,fx,totalCreditPnfs2GDP,totalCreditPnfsLCY,dsrPnfs,...,bankCreditPnfs_lag3,totalCreditPnfsLCY_lag3,totalCreditPnfs2GDP_lag3,cpi_yoy_lag4,bankCreditPnfs_lag4,totalCreditPnfsLCY_lag4,totalCreditPnfs2GDP_lag4,fx_std,date.1_y,is_systemic_crisis
0,DE,1948-10-01,1948-10-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,4.522,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DE,1949-01-01,1949-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,5.476,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DE,1949-04-01,1949-04-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,6.942,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DE,1949-07-01,1949-07-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,8.593,NaN,...,2.984000,4.522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE,1949-10-01,1949-10-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,10.573,NaN,...,3.614000,5.476,NaN,NaN,2.984000,4.522,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,DE,2023-04-01,2023-05-16 01:49:05.454545408,NaN,133.1571,6.796917,0.918634,125.1,5003.808,10.8,...,2115.533000,4926.568,129.4,6.494617,2128.664667,4817.403,128.4,0.089809,2023-04-01,0.0
299,DE,2023-07-01,2023-08-13 23:17:38.823529472,NaN,130.5617,5.832606,0.918989,123.5,5020.222,10.8,...,2215.088333,4957.597,128.1,7.110009,2115.533000,4926.568,129.4,0.043293,2023-07-01,0.0
300,DE,2023-10-01,2023-11-11 17:27:16.363636480,NaN,127.8992,3.648422,0.930387,122.0,5028.972,10.8,...,2243.533667,4972.174,126.3,8.676878,2215.088333,4957.597,128.1,0.053981,2023-10-01,0.0
301,DE,2024-01-01,2024-02-13 07:30:00.000000000,NaN,NaN,2.519272,0.921022,NaN,NaN,NaN,...,2251.737333,5003.808,125.1,8.406074,2243.533667,4972.174,126.3,0.023914,2024-01-01,0.0


In [6]:
data.columns

Index(['iso2', 'date', 'date.1_x', 'loansPnfs_yoy', 'resPropPrice', 'cpi_yoy',
       'fx', 'totalCreditPnfs2GDP', 'totalCreditPnfsLCY', 'dsrPnfs',
       'dsrHousehold', 'dsrNfc', 'spotRate10year', 'EAtermSpread',
       'spotRate2year', 'bankCreditPnfs', 'UStermSpread',
       'financialStressIndex', 'financialStressDummy', 'month',
       'cpi_yoy_growthRate', 'bankCreditPnfs_growthRate',
       'totalCreditPnfsLCY_growthRate', 'totalCreditPnfs2GDP_growthRate',
       'cpi_yoy_lag1', 'bankCreditPnfs_lag1', 'totalCreditPnfsLCY_lag1',
       'totalCreditPnfs2GDP_lag1', 'cpi_yoy_lag2', 'bankCreditPnfs_lag2',
       'totalCreditPnfsLCY_lag2', 'totalCreditPnfs2GDP_lag2', 'cpi_yoy_lag3',
       'bankCreditPnfs_lag3', 'totalCreditPnfsLCY_lag3',
       'totalCreditPnfs2GDP_lag3', 'cpi_yoy_lag4', 'bankCreditPnfs_lag4',
       'totalCreditPnfsLCY_lag4', 'totalCreditPnfs2GDP_lag4', 'fx_std',
       'date.1_y', 'is_systemic_crisis'],
      dtype='object')

In [7]:
cols = ['date','loansPnfs_yoy', 'resPropPrice', 'cpi_yoy',
       'fx', 'totalCreditPnfs2GDP', 'totalCreditPnfsLCY', 'dsrPnfs',
       'dsrHousehold', 'dsrNfc', 'spotRate10year', 'EAtermSpread',
       'spotRate2year', 'bankCreditPnfs', 'UStermSpread',
       'financialStressIndex', 'financialStressDummy', 'month',
       'cpi_yoy_growthRate', 'bankCreditPnfs_growthRate',
       'totalCreditPnfsLCY_growthRate', 'totalCreditPnfs2GDP_growthRate',
       'cpi_yoy_lag1', 'bankCreditPnfs_lag1', 'totalCreditPnfsLCY_lag1',
       'totalCreditPnfs2GDP_lag1', 'cpi_yoy_lag2', 'bankCreditPnfs_lag2',
       'totalCreditPnfsLCY_lag2', 'totalCreditPnfs2GDP_lag2', 'cpi_yoy_lag3',
       'bankCreditPnfs_lag3', 'totalCreditPnfsLCY_lag3',
       'totalCreditPnfs2GDP_lag3', 'cpi_yoy_lag4', 'bankCreditPnfs_lag4',
       'totalCreditPnfsLCY_lag4', 'totalCreditPnfs2GDP_lag4', 'fx_std',
       'is_systemic_crisis']

In [8]:
data['is_systemic_crisis'] = data['is_systemic_crisis'].fillna(0)
data = data[cols]
data = data.dropna(axis=1, how = 'all')

In [9]:
data = data[data['date']>'1960'].dropna(axis=1)

In [10]:
data.index = data['date']
data.drop('date', axis=1, inplace = True)

In [11]:
data.to_csv("./Thomas/data_export.csv")